# 1. Importing modules and functions

In [27]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import chembl_structure_pipeline
from molvs import standardize_smiles
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Categorical, Integer, Continuous
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

# 2.Data entry and curation work set

In [2]:
uploaded_file_ws="datasets/HDAC3_work.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  1400 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [3]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [4]:
len(y_tr)

1400

# 3.Standardization SDF file for work set

In [5]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  1400 molecules


# 4.Data entry and curation test set

In [6]:
uploaded_file_ts="datasets/HDAC3_test.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  351 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [7]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [8]:
len(y_ts)

351

# 5.Standardization SDF file for test set

In [9]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  351 molecules


# 6.Calculation MorganFingerprint for work set

In [10]:
fp_tr = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2,nBits=1024,useFeatures=False,useChirality = False) for m in moldf_ws]

In [11]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [12]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [13]:
x_tr.shape

(1400, 1024)

# 7.Calculation MorganFingerprint for test set

In [14]:
fp_ts = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2,nBits=1024,useFeatures=False,useChirality = False) for m in moldf_ts]

In [15]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [16]:
x_ts = rdkit_numpy_convert(fp_ts)

In [17]:
x_ts.shape

(351, 1024)

In [18]:
type(x_tr)

numpy.ndarray

In [19]:
x_tr

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [20]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

In [21]:
y_tr

array([ 4.01,  4.04,  4.05, ..., 10.03, 10.06, 10.1 ], dtype=float32)

 ## GradientBoostingRegressor model building and validation

In [22]:
seed = 42

In [23]:
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [24]:
param_grid = {'learning_rate': Continuous(0.01, 0.04),
                  'subsample': Continuous(0.5, 0.9),
                  'n_estimators': Integer(100, 2000),
                  'max_depth': Integer(4, 10)
                 }

In [25]:
evolved_estimator= GASearchCV(estimator=GradientBoostingRegressor(),
                              cv=cv,
                              scoring='r2',
                              param_grid=param_grid,
                              n_jobs=-1,
                              verbose=True,
                              population_size=10,
                              generations=10)

In [26]:
%%time
evolved_estimator.fit(x_tr, y_tr)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.669492	0.0124053  	0.680436   	0.64039    
1  	20    	0.67801 	0.00481094 	0.681994   	0.668008   
2  	20    	0.681688	0.00187179 	0.684746   	0.678931   
3  	20    	0.684023	0.0022919  	0.686789   	0.679323   
4  	20    	0.684922	0.000773341	0.686789   	0.683464   
5  	20    	0.685407	0.000960994	0.687711   	0.684746   
6  	20    	0.685559	0.000707408	0.686789   	0.684746   
7  	20    	0.685739	0.000761736	0.687126   	0.685121   
8  	20    	0.686405	0.000618116	0.687126   	0.685154   
9  	20    	0.686343	0.000467866	0.687126   	0.685921   
10 	20    	0.686852	0.00132001 	0.689097   	0.684645   
CPU times: total: 1min 40s
Wall time: 3h 4min 35s


GASearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
           estimator=GradientBoostingRegressor(learning_rate=0.012188222156897334,
                                               max_depth=10, n_estimators=1029,
                                               subsample=0.5725587175671426),
           generations=10, n_jobs=-1,
           param_grid={'learning_rate': <sklearn_genetic.space.space.Continuous object at 0x0000017679539840>,
                       'max_depth': <sklearn_genetic.space.space.Integer object at 0x000001767953BEB0>,
                       'n_estimators': <sklearn_genetic.space.space.Integer object at 0x000001767953B970>,
                       'subsample': <sklearn_genetic.space.space.Continuous object at 0x000001767953B370>},
           population_size=10, return_train_score=True, scoring='r2',
           warm_start_configs=[])

In [28]:
# Best parameters found
print(evolved_estimator.best_params_)

{'learning_rate': 0.012188222156897334, 'subsample': 0.5725587175671426, 'n_estimators': 1029, 'max_depth': 10}


## Comparing with GridSearchCV

In [29]:
param_grid = {'learning_rate': [0.01, 0.04],
                  'subsample'    : [0.9, 0.5],
                  'n_estimators' : [100,1000, 2000],
                  'max_depth'    : [4, 10]
                 }

In [30]:
m = GridSearchCV(GradientBoostingRegressor(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [31]:
%%time
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
CPU times: total: 1min 19s
Wall time: 1h 20min 3s


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=GradientBoostingRegressor(), n_jobs=2,
             param_grid={'learning_rate': [0.01, 0.04], 'max_depth': [4, 10],
                         'n_estimators': [100, 1000, 2000],
                         'subsample': [0.9, 0.5]},
             verbose=1)

In [32]:
best_GBR = m.best_estimator_

In [33]:
m.best_params_

{'learning_rate': 0.01,
 'max_depth': 10,
 'n_estimators': 2000,
 'subsample': 0.5}

In [38]:
y_pred_CV_GBR = cross_val_predict(best_GBR, x_tr, y_tr, cv=cv, n_jobs=-1)

In [39]:
y_pred_CV_GBR

array([5.53896186, 6.02256925, 4.80940266, ..., 9.54505482, 7.15069989,
       9.73646898])

In [40]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_GBR), 2)
Q2_CV

0.69

In [41]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_GBR)), 2)
RMSE_CV

0.67